In [1]:
%pip install pypdf cohere tiktoken langchain sentence_transformers  pinecone -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install -U langchain-google-genai pillow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install pinecone version==3.0.3 langchain_pinecone version==0.0.3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement version==3.0.3 (from versions: 0.1.0, 0.1.1)

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for version==3.0.3


In [5]:
import os
from pathlib import Path
import pinecone
from pprint import pprint

from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate

c:\Users\samit\Desktop\Vector_db_final\venv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = "./stock.pdf"  
loader = PyPDFLoader(file_path)
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True 
)
all_splits = text_splitter.split_documents(documents)

print(f"Document split into {len(all_splits)} chunks.")
print("\n--- Example of the first chunk: ---")
print(all_splits[0].page_content)

Document split into 24 chunks.

--- Example of the first chunk: ---
Stock Market Performance in 2024
U.S. Market Overview
The year 2024 was a remarkably strong one for equities, with the U.S. stock market extending the
robust gains seen in the prior year. The benchmark S&P 500 index delivered roughly a 25% total
return for 2024 (around +23% in price terms)
. This marked the second consecutive year of over 20% returns for the S&P 500 - a feat not
observed since the late 1990s
. The tech-heavy Nasdaq Composite outpaced the broader market, jumping nearly 29% for the year
. In contrast, smaller-cap stocks had more modest performance: the S&P 500 Equal-Weight index
and the Russell 2000 (small-cap benchmark) each rose about 10-11% in 2024
. The disparity underscored that 2024's rally was not evenly distributed across the market. A key
theme was the dominance of mega-cap technology stocks. A group of seven powerhouse
companies often dubbed the "Magnificent 7" - Apple, Microsoft, Alphabet (Goo

In [7]:
from langchain_pinecone import PineconeEmbeddings
import os

model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=os.environ.get('PINECONE_API_KEY')
)


In [8]:
from pinecone import Pinecone, ServerlessSpec
import time
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
print(f"API Key Used: {pinecone_api_key}") 
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

cloud ='aws'
region ='us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)

index_name = "rag-getting-started"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=spec
    )

# See that it is empty
print("Index before upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")

API Key Used: pcsk_nHxB4_5vsNeKLQNU8HgfEsFndeu1AXL7uBgG5SXdcfLGKMmt2VdwR7LzHQ8JA4xWX1Gxz
Index before upsert:
{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}




In [ ]:
from langchain_pinecone import PineconeVectorStore

namespace = "wondervector5000"

docsearch = PineconeVectorStore.from_documents(
    documents=all_splits ,
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

time.sleep(5)

print("Index after upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")
time.sleep(2)

Index after upsert:
{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wondervector5000': {'vector_count': 24}},
 'total_vector_count': 24,
 'vector_type': 'dense'}




In [17]:
index = pc.Index(index_name)
namespace = "wondervector5000"

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0], 
        namespace=namespace, 
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)
    print("\n")



{'matches': [{'id': '0d13d1fa-156e-43f6-a0f9-019c733ad909',
              'metadata': {'creationdate': 'D:20250411184712',
                           'creator': 'PyPDF',
                           'page': 3.0,
                           'page_label': '4',
                           'producer': 'PyFPDF 1.7.2 '
                                       'http://pyfpdf.googlecode.com/',
                           'source': './stock.pdf',
                           'start_index': 0.0,
                           'text': 'than the stock price. Even at 40× '
                                   "earnings, Amazon's valuation was still on "
                                   'the higher side\n'
                                   'compared to the broader market, but '
                                   'investors seemed comfortable paying a '
                                   'premium for its\n'
                                   'dominant market positions. Overall, 2024 '
                           

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
namespace = "wondervector5000"  

retriever = docsearch.as_retriever(
    search_kwargs={'namespace': namespace}
)


llm = ChatGoogleGenerativeAI(
    google_api_key=os.environ.get('GOOGLE_API_KEY'),
    model='gemini-2.5-pro',
    temperature=0.0
)

combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

c:\Users\samit\Desktop\Vector_db_final\venv2\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [29]:
query1 = "What are the first 3 steps for getting started with the WonderVector5000?"

query2 = "tell me about what is this stock pdf sbout?"

In [22]:
answer1_without_knowledge = llm.invoke(query1)

print("Query 1:", query1)
print("\nAnswer without knowledge:\n\n", answer1_without_knowledge.content)
print("\n")
time.sleep(2)

Query 1: What are the first 3 steps for getting started with the WonderVector5000?

Answer without knowledge:

 Of course! Congratulations on your new (and highly advanced) WonderVector5000. Getting started is an exciting process.

Here are the essential first 3 steps to get you up and running:

### Step 1: Unbox and Power Up
First, carefully unbox your device. Inside, you should find the WonderVector5000 unit, a quantum power adapter, and a neural-link cable.

*   **Inspect:** Check all components for any signs of damage during shipping.
*   **Connect Power:** Plug the quantum power adapter into a wall outlet and connect it to the designated port on the back of the WonderVector5000.
*   **Power On:** Press and hold the glowing activation button on the top of the unit for three seconds. You should see the front display light up with the WonderVector welcome animation.

### Step 2: Initial Configuration and Network Connection
Once powered on, the device will guide you through its initia

In [23]:
answer1_with_knowledge = retrieval_chain.invoke({"input": query1})

print("Answer with knowledge:\n\n", answer1_with_knowledge['answer'])
print("\nContext used:\n\n", answer1_with_knowledge['context'])
print("\n")
time.sleep(2)

Answer with knowledge:

 Based on the context provided, there is no information about the WonderVector5000 or any steps for getting started with it.

Context used:

 []




In [24]:
answer2_without_knowledge = llm.invoke(query2)

print("Query 2:", query2)
print("\nAnswer without knowledge:\n\n", answer2_without_knowledge.content)
print("\n")
time.sleep(2)

Query 2: tell me about stocks of apple in 2024?

Answer without knowledge:

 Of course. Here is a detailed overview of Apple's stock (ticker: AAPL) in 2024, covering its performance, the key factors driving it, and the general outlook.

**Executive Summary:**

In 2024, Apple stock has faced a notable shift in sentiment. After years of being a market leader, it has underperformed the broader market (like the S&P 500 and Nasdaq 100) for the first time in a while. The narrative has moved from "unstoppable growth" to one of caution, driven by concerns about slowing iPhone sales, intense competition in China, and significant regulatory pressures. However, the company's massive profitability, loyal customer base, and upcoming AI strategy remain powerful long-term strengths.

---

### 1. Stock Performance in 2024 (Year-to-Date)

*   **Underperformance:** Unlike the "Magnificent 7" stocks like NVIDIA and Meta, which have soared, Apple's stock has been relatively flat or down for much of 2024.


In [30]:
answer2_with_knowledge = retrieval_chain.invoke({"input": query2})

print("\nAnswer with knowledge:\n\n", answer2_with_knowledge['answer'])
print("\nContext Used:\n\n", answer2_with_knowledge['context'])
print("\n")
time.sleep(2)


Answer with knowledge:

 Based on the context provided, this document is about the performance of the U.S. stock market in 2024. It covers:

*   **Overall Market Performance:** The S&P 500 had a total return of about 25%, and the tech-heavy Nasdaq jumped nearly 29%. The rally was largely driven by mega-cap technology stocks.
*   **Tesla (TSLA):** The stock ended 2024 up approximately 63% despite a volatile year. It was trading at a very high multiple of over 100 times its earnings.
*   **Meta:** The stock gained 72%, indicating a return to favor for social media and online advertising companies.
*   **Palantir (PLTR):** This was the single best-performing stock in the S&P 500 for the year, soaring by about 340% due to achieving profitability and excitement over its AI platforms.

Context Used:

 [Document(id='ae25d0ae-4d4b-44eb-ad7f-dade80a2240e', metadata={'creationdate': 'D:20250411184712', 'creator': 'PyPDF', 'page': 5.0, 'page_label': '6', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.g

In [32]:
%pip install gradio

  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached starlette-0.47.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
   ---------------------------------------- 0.0/59.5 MB ? eta -:--:--
    --------------------------------------- 1.0/59.5 MB 6.3 MB/s eta 0:00:10
   - -------------------------------------- 1.8/59.5 MB 4.8 MB/s eta 0:00:13
   - -------------------------------------- 2.6/59.5 MB 4.2 MB/s eta 0:00:14
   -- -------------------


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import gradio as gr
import time

# 1. Define the function that will power the Gradio interface
def get_rag_response(query):
    """
    Takes a user query, invokes the RAG chain, and returns the
    answer and the context used.
    """
    print(f"Received query: {query}")
    result = retrieval_chain.invoke({"input": query})
    
    # Format the context for better display
    context_text = ""
    if 'context' in result and result['context']:
        context_text = "\n\n---\n\n".join([doc.page_content for doc in result['context']])
    else:
        context_text = "No context was found for this query."
        
    return result['answer'], context_text

# 2. Create the Gradio interface
iface = gr.Interface(
    fn=get_rag_response,
    inputs=gr.Textbox(label="Your Question"),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Context Used")
    ],
    title="📄 RAG with Pinecone, LangChain, and Gemini",
    description="Ask a question about your documents. The application will retrieve relevant context from Pinecone and use Gemini to generate an answer."
)

# 3. Launch the app
iface.launch(debug=True) # debug=True allows you to see print statements in your console

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Received query: can you tell me about apple stocks in 2024
Created dataset file at: .gradio\flagged\dataset1.csv
Keyboard interruption in main thread... closing server.
